# 📊 SentimentAPI — Análisis de Sentimientos de Feedbacks

**Sector:** Atención al cliente / Marketing / Operaciones  
**Objetivo:** Clasificar comentarios de usuarios en **positivo** vs **negativo (neutral + negativo)** para generar insights accionables y permitir consumo desde backend vía API.  
**Equipo:** Data Science (Google Colab + Python)  
**Repositorio:** `https://github.com/ElvisG2003/Sentiment-API-Hackathon.git`

---

## ✅ Resultado esperado (Deliverables)

Al finalizar este notebook deberíamos tener:

- **Notebook reproducible end-to-end**  
  *(EDA → preparación → entrenamiento → evaluación → exportación)*
- **Modelo baseline:** TF-IDF + Logistic Regression *(scikit-learn)*
- **Métricas reportadas:** Accuracy / Precision / Recall / F1-score + Matriz de confusión
- **Exportación del pipeline:**  
  `joblib.dump(pipeline, "sentiment_pipeline.joblib")` *(listo para FastAPI / backend)*

---

## 🧭 Convenciones del notebook (cómo trabajamos)

Cada sección debe incluir:

1) **Qué haremos** (qué proceso/celdas se ejecutan)  
2) **Por qué** (qué valida o qué problema resuelve)  
3) **Qué esperamos ver** (output “correcto” y señales de alerta)

Reglas del proyecto:

- **No modificamos el dataset original:** trabajamos con copias (`df_core`, `df_model`).
- **Neutral se mantiene en el dataset** (no se elimina).  
  Para el baseline, se mapea a `0` junto con negativo.
- Etiquetas finales del modelo:
  - `1` = positivo  
  - `0` = neutral o negativo  

---

## 🧪 Validaciones mínimas obligatorias (antes de confiar en métricas)

- Distribución de clases (labels)  
- Variedad real del texto (texts únicos, duplicados y “plantillas”)  
- Split correcto (train/test estratificado)  
- Prueba anti-fuga: **label shuffle test** *(debería caer a rendimiento cercano a azar)*

---

## 🧱 Pipeline baseline (resumen técnico)

**Entrada:** texto del cliente (tweet/feedback)  
**Preprocesamiento:** limpieza liviana + TF-IDF (1–2 ngramas)  
**Modelo:** Logistic Regression  
**Salida:** predicción binaria `0/1` + (opcional) probabilidad de positivo

> **Nota:** Este notebook define el baseline reproducible. Mejoras futuras podrían incluir balance de clases, ajuste de umbral, modelos más robustos y evaluación cross-domain.


## 0) Setup del entorno

### Qué haremos
Instalar librerías mínimas para poder leer datasets desde Hugging Face usando rutas `hf://...`.

### Por qué
En Google Colab, a veces el soporte para `hf://` no viene habilitado por defecto.

### Qué esperamos ver
La instalación termina sin errores (si sale “already satisfied”, también está bien).


In [ ]:
!pip -q install huggingface_hub fsspec

## 1) Carga del dataset

### Qué haremos
Leer el CSV desde Hugging Face y ver:
- cantidad de filas/columnas (`shape`)
- primeras filas (`head()`)

### Por qué
Antes de entrenar cualquier cosa, necesitamos confirmar que:
- el archivo se cargó bien
- existen columnas de texto y de etiqueta (sentiment)

### Qué esperamos ver
- Miles de filas (no 25.000 con 15 textos únicos como antes)
- Columnas que incluyan `text` y `airline_sentiment` (o similares)


In [ ]:
import pandas as pd

df = pd.read_csv("hf://datasets/osanseviero/twitter-airline-sentiment/Tweets.csv")

print("✅ Dataset Cargado")
print("Forma (rows, cols):", df.shape)
df.head(3)


✅ Dataset Cargado
Forma (rows, cols): (14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)


## 2) Inspección de columnas

### Qué haremos
Listar todas las columnas disponibles.

### Por qué
Para saber exactamente:
- dónde está el texto del tweet
- dónde está la etiqueta (sentiment)
- qué columnas extras hay (a veces sirven para análisis)

### Qué esperamos ver
Una lista donde aparezcan columnas como `text` y `airline_sentiment`.


In [ ]:
print("Columnas:")
for c in df.columns:
    print("-", c)


Columnas:
- tweet_id
- airline_sentiment
- airline_sentiment_confidence
- negativereason
- negativereason_confidence
- airline
- airline_sentiment_gold
- name
- negativereason_gold
- retweet_count
- text
- tweet_coord
- tweet_created
- tweet_location
- user_timezone


## 3) Detectar columnas clave (texto y label)

### Qué haremos
Intentar detectar automáticamente:
- `text_col`: columna donde está el texto
- `label_col`: columna donde está el sentimiento

### Por qué
Evita errores típicos como entrenar con la columna incorrecta.

### Qué esperamos ver
Que detecte:
- `text_col = text`
- `label_col = airline_sentiment`
(pero si el dataset trae nombres distintos, también lo detectará)


In [ ]:
possible_text_cols = ["text", "tweet", "content", "message", "review"]
possible_label_cols = ["airline_sentiment", "sentiment", "label", "target", "class"]

text_col = next((c for c in possible_text_cols if c in df.columns), None)
label_col = next((c for c in possible_label_cols if c in df.columns), None)

print("Columna de texto detectada:", text_col)
print("Fila de label detectada:", label_col)

assert text_col is not None, "❌ No se encontró automáticamente la columna de texto."
assert label_col is not None, "❌ No se encontró automáticamente la columna de etiqueta (sentiment)."


Columna de texto detectada: text
Fila de label detectada: airline_sentiment


## 4) Sanity checks del texto (vacíos/nulos)

### Qué haremos
- Convertir la columna de texto a string
- Contar cuántos textos están vacíos

### Por qué
Si hay textos vacíos, el modelo aprende “nada” y puede distorsionar métricas.

### Qué esperamos ver
- “Texto vacio” idealmente cercano a 0 (o bajo)
- Nulos en otras columnas no son problema si no las usamos


In [ ]:
texts_raw = df[text_col].fillna("").astype(str)

print("Columnas totales:", len(texts_raw))
print("Texto vacio:", (texts_raw.str.strip() == "").sum())

print("\nValores faltantes (Top 10 columnas):")
display(df.isna().sum().sort_values(ascending=False).head(10))


Columnas totales: 14640
Texto vacio: 0

Valores faltantes (Top 10 columnas):


,0
negativereason_gold,14608
airline_sentiment_gold,14600
tweet_coord,13621
negativereason,5462
user_timezone,4820
tweet_location,4733
negativereason_confidence,4118
airline,0
tweet_id,0
airline_sentiment,0


## 5) Distribución de etiquetas (labels)

### Qué haremos
Contar cuántos ejemplos hay de:
- negative
- neutral
- positive

### Por qué
Necesitamos confirmar que el dataset:
- tiene neutral de verdad (no 0)
- no está roto (por ejemplo, solo una clase)

### Qué esperamos ver
Tres clases presentes, con porcentajes razonables.
(Es normal que `negative` sea la clase más grande en soporte al cliente.)


In [ ]:
print("Labels contadas:")
display(df[label_col].value_counts())

print("\nLabel en porcentaje:")
display((df[label_col].value_counts(normalize=True) * 100).round(2))


Labels contadas:


,count
airline_sentiment,
negative,9178
neutral,3099
positive,2363



Label en porcentaje:


,proportion
airline_sentiment,
negative,62.69
neutral,21.17
positive,16.14


## 6) Variedad del texto (¿es un dataset real o plantillado?)

### Qué haremos
Medir:
- cuántos textos únicos hay
- cuántos duplicados hay
- qué tanto “dominan” los top-10 textos más repetidos

### Por qué
El dataset anterior falló porque tenía 25.000 filas pero solo 15 textos únicos.
Eso hace que el modelo “memorice” y las métricas sean falsas.

### Qué esperamos ver
- miles de textos únicos (no 15)
- que los top-10 no cubran un porcentaje enorme (idealmente bajo)


In [ ]:
unique_texts = texts_raw.nunique()
dup_count = len(texts_raw) - unique_texts

print("Texto unico:", unique_texts)
print("Texto duplicado:", dup_count)

top10 = texts_raw.value_counts().head(10)
print("\nTop 10 texto mas frecuente:")
display(top10)

coverage = top10.sum() / len(texts_raw)
print(f"\nPorcentaje del top 10: {coverage:.2%}")


Texto unico: 14427
Texto duplicado: 213

Top 10 texto mas frecuente:


,count
text,
@united thanks,6
@AmericanAir thanks,5
@JetBlue thanks!,5
@SouthwestAir sent,5
@united thank you!,4
@AmericanAir thank you!,4
@USAirways thank you,3
@united thank you,3
@USAirways thanks,3



Porcentaje del top 10: 0.28%


## 7) Limpieza liviana y repetición real

### Qué haremos
Crear una versión `texto_clean` donde:
- pasamos a minúsculas
- removemos URLs
- normalizamos espacios

Luego repetimos la medición de duplicados.

### Por qué
A veces muchos tweets parecen únicos solo porque cambian los links.
Si removemos URLs, medimos repetición “real”.

### Qué esperamos ver
- La variedad sigue siendo alta (miles de textos únicos)
- Coverage top-10 clean no se dispara demasiado


In [ ]:
import re

def clean_tweet(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"http\S+|www\.\S+", " ", s)  # Remueve URL
    s = re.sub(r"@\w+", " ", s)  # Removemos los @
    s = re.sub(r"\s+", " ", s).strip()      # Normalizamos
    return s

texts_clean = texts_raw.apply(clean_tweet)

print("Texto vacio limpiado:", (texts_clean.str.strip() == "").sum())
print("Textos unicos limpiados:", texts_clean.nunique())

top10_clean = texts_clean.value_counts().head(10)
print("\nTop 10 textos mas frecuentes limpiados:")
display(top10_clean)

coverage_clean = top10_clean.sum() / len(texts_clean)
print(f"\nPorcentaje del top 10 limpiado: {coverage_clean:.2%}")


Texto vacio limpiado: 0
Textos unicos limpiados: 14276

Top 10 textos mas frecuentes limpiados:


,count
text,
thanks!,24
thank you!,23
thanks,18
thank you,15
thank you.,11
's ceo battles to appease passengers and wall street - waterbury republican american,9
thanks so much!,8
sent,8
done,7



Porcentaje del top 10 limpiado: 0.87%


## 8) Construir df_core (formato proyecto)

### Qué haremos
Crear un dataframe mínimo con columnas consistentes para el proyecto:

- `id_cliente`: id (si existe `tweet_id`, lo usamos; si no, usamos el índice)
- `texto_de_review`: texto original (sin limpiar)
- `texto_clean`: texto limpio (para entrenar)
- `opinion_raw`: etiqueta original (negative/neutral/positive)

### Por qué
Esto deja el pipeline ordenado y evita tocar el dataframe original (`df`).

### Qué esperamos ver
Un df con 4 columnas y la misma cantidad de filas que el dataset original.


In [ ]:
df_core = df.copy()

df_core["id_cliente"] = df_core["tweet_id"] if "tweet_id" in df_core.columns else df_core.index
df_core["texto_de_review"] = texts_raw
df_core["texto_clean"] = texts_clean
df_core["opinion_raw"] = df_core[label_col].astype(str)

df_core = df_core[["id_cliente", "texto_de_review", "texto_clean", "opinion_raw"]].copy()

print("✅ df_core ready:", df_core.shape)
df_core.head(3)


✅ df_core ready: (14640, 4)


,id_cliente,texto_de_review,texto_clean,opinion_raw
0,570306133677760513,@VirginAmerica What @dhepburn said.,what said.,neutral
1,570301130888122368,@VirginAmerica plus you've added commercials t...,plus you've added commercials to the experienc...,positive
2,570301083672813571,@VirginAmerica I didn't today... Must mean I n...,i didn't today... must mean i need to take ano...,neutral


## 9) Crear etiqueta numérica del proyecto (opinion)

### Qué haremos
Mapear la etiqueta de 3 clases a binaria según la política del proyecto:

- positive → 1
- neutral → 0
- negative → 0

### Por qué
Nuestro baseline actual es binario:
“positivo” vs “negativo”.

### Qué esperamos ver
- 0 NaNs después del mapping
- distribución de `opinion` con 0 y 1


In [ ]:
map_bin = {"positive": 1, "neutral": 0, "negative": 0}

df_core["opinion"] = df_core["opinion_raw"].map(map_bin)

print("NaNs after mapping:", df_core["opinion"].isna().sum())
display(df_core["opinion"].value_counts())

NaNs after mapping: 0


,count
opinion,
0,12277
1,2363


In [ ]:
df_core.shape

(14640, 5)

## 10) Dataset final para modelado (df_model)

### Qué haremos
1) Crear `df_model` a partir de `df_core`.
2) Deduplicar por `texto_clean` (y opcionalmente por `opinion`).
3) Confirmar distribución de clases final.

### Por qué
- En datasets con textos repetidos, el modelo puede “memorizar” frases y dar métricas infladas.
- Deduplicar reduce leakage y mejora generalización.

### Qué esperamos ver
- Menos filas que `df_core` (si había repetidos).
- Distribución de clases (0/1) razonable.


In [ ]:
# ==========================
# 10) Buildeamos df_model
# ==========================

# Mantenemos solo las columnas necesarias
df_model = df_core[["id_cliente", "texto_de_review", "texto_clean", "opinion_raw", "opinion"]].copy()

before = len(df_model)

# Deduplicamos los clean texts exactos (y label) para reducir escape
df_model = df_model.drop_duplicates(subset=["texto_clean", "opinion"]).copy()

after = len(df_model)

print("Filas antes:", before)
print("Filas despues :", after)
print("Eliminados  :", before - after)

print("\nDistribucion binaria de label (opinion):")
display(df_model["opinion"].value_counts())

print("\nEjemplos:")
display(df_model[["texto_de_review", "texto_clean", "opinion_raw", "opinion"]].sample(5, random_state=42))


Filas antes: 14640
Filas despues : 14304
Eliminados  : 336

Distribucion binaria de label (opinion):


,count
opinion,
0,12064
1,2240



Ejemplos:


,texto_de_review,texto_clean,opinion_raw,opinion
9984,@USAirways really nigga.. Ur a fuck boy,really nigga.. ur a fuck boy,negative,0
4548,@SouthwestAir @Imaginedragons I tried. 😔 It's ...,i tried. 😔 it's okay,neutral,0
1020,@united ok. I just submitted. Thanks for the o...,ok. i just submitted. thanks for the opportuni...,positive,1
7580,@JetBlue here's part: was at bag drop inserted...,here's part: was at bag drop inserted my jetbl...,negative,0
5226,@SouthwestAir flight to PHL from FLL Cancelled...,flight to phl from fll cancelled flighted 2/21...,negative,0


## 11) Split train/test (estratificado)

### Qué haremos
Separar el dataset en:
- Train: el modelo aprende
- Test: medimos desempeño con datos "nuevos"

Usaremos `stratify=y` para que la proporción de clases (0/1) sea similar en train y test.

### Por qué
Sin estratificación, podrías terminar con un test con casi puros 0 o puros 1, y las métricas no sirven.

### Qué esperamos ver
- Train ~80%
- Test ~20%
- Distribución de clases muy parecida en ambos


In [ ]:
from sklearn.model_selection import train_test_split

X = df_model["texto_clean"].astype(str)
y = df_model["opinion"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Tamaño de Train:", len(X_train))
print("Tamaño de Test :", len(X_test))

print("\nTrain label %:")
display((y_train.value_counts(normalize=True) * 100).round(2))

print("\nTest label %:")
display((y_test.value_counts(normalize=True) * 100).round(2))


Tamaño de Train: 11443
Tamaño de Test : 2861

Train label %:


,proportion
opinion,
0,84.34
1,15.66



Test label %:


,proportion
opinion,
0,84.34
1,15.66


## 12) Entrenar modelo baseline (TF-IDF + Logistic Regression)

### Qué haremos
Entrenar un pipeline con:
- TF-IDF (convierte texto a números)
- Logistic Regression (clasificador lineal)

### Por qué
Es un baseline fuerte, rápido y fácil de explicar. Perfecto para hackathon.

### Qué esperamos ver
- Entrenamiento sin errores
- Un pipeline listo para predecir con `.predict()`


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words="english")),
    ("clf", LogisticRegression(max_iter=1000))
])

pipeline.fit(X_train, y_train)

print("✅ Entrenamiento listo")


✅ Entrenamiento listo


## 13) Evaluación (métricas + matriz de confusión)

### Qué haremos
Evaluar el baseline usando:
- Accuracy
- Precision
- Recall
- F1-score
- Confusion Matrix

### Por qué
Nos permite entender si el modelo:
- acierta de verdad
- o está sesgado (por ejemplo, predice todo como 0)

### Qué esperamos ver
- Métricas razonables (no necesariamente perfectas)
- Matriz de confusión con aciertos en ambas clases (0 y 1)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

y_pred = pipeline.predict(X_test)

print("Exactitud :", accuracy_score(y_test, y_pred))
print("Precisión:", precision_score(y_test, y_pred, zero_division=0))
print("Llamada   :", recall_score(y_test, y_pred, zero_division=0))
print("F1-score :", f1_score(y_test, y_pred, zero_division=0))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

print("\nReporte de Clasificacion:\n")
print(classification_report(y_test, y_pred, digits=4, zero_division=0))


Exactitud : 0.8996854246766864
Precisión: 0.851528384279476
Llamada   : 0.43526785714285715
F1-score : 0.5760709010339734

Confusion Matrix:
 [[2379   34]
 [ 253  195]]

Reporte de Clasificacion:

              precision    recall  f1-score   support

           0     0.9039    0.9859    0.9431      2413
           1     0.8515    0.4353    0.5761       448

    accuracy                         0.8997      2861
   macro avg     0.8777    0.7106    0.7596      2861
weighted avg     0.8957    0.8997    0.8856      2861



## 14) Sanity check anti-fuga (Label Shuffle Test)

### Qué haremos
Entrenar el mismo pipeline pero con las etiquetas de entrenamiento mezcladas al azar.

### Por qué
Si el modelo sigue sacando resultados altos con labels aleatorias, significa que:
- hay fuga de datos (leakage), o
- estamos evaluando mal.

### Qué esperamos ver
La accuracy debería bajar cerca de “azar” / clase mayoritaria.


In [ ]:
from sklearn.metrics import accuracy_score

y_train_shuf = y_train.sample(frac=1, random_state=42).values

pipeline_shuf = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words="english")),
    ("clf", LogisticRegression(max_iter=1000))
])

pipeline_shuf.fit(X_train, y_train_shuf)
pred_shuf = pipeline_shuf.predict(X_test)

print("Exactitud con labels random:", accuracy_score(y_test, pred_shuf))


Exactitud con labels random: 0.8434113946172667


## A) Modelo mejorado para subir recall de positivos (class_weight="balanced")

### Qué haremos
Entrenar el mismo baseline (TF-IDF + Logistic Regression) pero usando `class_weight="balanced"`.

### Por qué
Nuestro dataset está desbalanceado (muchos 0 y pocos 1).  
Esto hace que el modelo tienda a predecir 0 para “no equivocarse”.  
Con `balanced`, los errores en la clase 1 pesan más → el modelo intenta detectar más positivos.

### Qué esperamos ver
- Recall de la clase 1 sube.
- Precision de la clase 1 puede bajar un poco.


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

pipeline_bal = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=5000, ngram_range=(1,2), stop_words="english")),
    ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

pipeline_bal.fit(X_train, y_train)
y_pred_bal = pipeline_bal.predict(X_test)

print("=== Modelo Balanceado (limite=0.50) ===")
print("Exactitud :", accuracy_score(y_test, y_pred_bal))
print("Precisión:", precision_score(y_test, y_pred_bal, zero_division=0))
print("Re-call   :", recall_score(y_test, y_pred_bal, zero_division=0))
print("F1-score :", f1_score(y_test, y_pred_bal, zero_division=0))
print("\nMatrix de Confusión:\n", confusion_matrix(y_test, y_pred_bal))
print("\nReporte:\n", classification_report(y_test, y_pred_bal, digits=4, zero_division=0))


=== Modelo Balanceado (limite=0.50) ===
Exactitud : 0.8864033554701154
Precisión: 0.6047700170357752
Re-call   : 0.7924107142857143
F1-score : 0.6859903381642513

Matrix de Confusión:
 [[2181  232]
 [  93  355]]

Reporte:
               precision    recall  f1-score   support

           0     0.9591    0.9039    0.9307      2413
           1     0.6048    0.7924    0.6860       448

    accuracy                         0.8864      2861
   macro avg     0.7819    0.8481    0.8083      2861
weighted avg     0.9036    0.8864    0.8923      2861



## B) Ajuste de umbral (threshold) para controlar Precision vs Recall

### Qué haremos
Calcular probabilidades y probar distintos umbrales (0.50, 0.45, 0.40, 0.35, 0.30).

### Por qué
Queremos mejorar recall de positivos:
- Umbral alto (0.50): más conservador (menos positivos detectados)
- Umbral bajo (0.35): detecta más positivos (sube recall) pero puede aumentar falsos positivos

### Qué esperamos ver
- Al bajar el umbral: recall sube
- Precision puede bajar
- Elegimos el umbral que cumpla el objetivo del proyecto


In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

proba_pos = pipeline_bal.predict_proba(X_test)[:, 1]

thresholds = [0.50, 0.45, 0.40, 0.35, 0.30, 0.25]
results = []

print("TH   | Precision  Recall  F1")
print("-------------------------------")
for th in thresholds:
    pred_th = (proba_pos >= th).astype(int)
    prec = precision_score(y_test, pred_th, zero_division=0)
    rec  = recall_score(y_test, pred_th, zero_division=0)
    f1   = f1_score(y_test, pred_th, zero_division=0)
    results.append((th, prec, rec, f1))
    print(f"{th:0.2f} |   {prec:0.3f}    {rec:0.3f}  {f1:0.3f}")


TH   | Precision  Recall  F1
-------------------------------
0.50 |   0.605    0.792  0.686
0.45 |   0.552    0.830  0.663
0.40 |   0.508    0.857  0.638
0.35 |   0.451    0.882  0.597
0.30 |   0.387    0.924  0.545
0.25 |   0.343    0.946  0.504


## Evaluación final con threshold elegido

### Qué haremos
Aplicar el threshold elegido sobre las probabilidades (`predict_proba`) y calcular:
- matriz de confusión
- classification report

### Por qué
Este será el comportamiento real que usará la API (no el `.predict()` default).

### Qué esperamos ver
Recall(1) alto (más positivos detectados) y entender cuántos falsos positivos aparecen.


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

chosen_th = 0.40  # Es preferible que backend revise falsos positivos antes que perder reales

y_pred_final = (proba_pos >= chosen_th).astype(int)

print("Limite:", chosen_th)
print("Exactitud:", accuracy_score(y_test, y_pred_final))
print("Matrix de Confusión:\n", confusion_matrix(y_test, y_pred_final))
print("\nReport:\n", classification_report(y_test, y_pred_final, digits=4, zero_division=0))


Limite: 0.4
Exactitud: 0.8476057322614471
Matrix de Confusión:
 [[2041  372]
 [  64  384]]

Report:
               precision    recall  f1-score   support

           0     0.9696    0.8458    0.9035      2413
           1     0.5079    0.8571    0.6379       448

    accuracy                         0.8476      2861
   macro avg     0.7388    0.8515    0.7707      2861
weighted avg     0.8973    0.8476    0.8619      2861



In [ ]:
import numpy as np

#Revision de positivos, si son demasiados, backend/UI se puede complicar

for th in [0.50, 0.45, 0.40, 0.35]:
    pred = (proba_pos >= th).astype(int)
    print(f"TH={th:.2f} -> predicted positives: {pred.sum()} / {len(pred)} ({pred.mean():.2%})")


TH=0.50 -> predicted positives: 587 / 2861 (20.52%)
TH=0.45 -> predicted positives: 674 / 2861 (23.56%)
TH=0.40 -> predicted positives: 756 / 2861 (26.42%)
TH=0.35 -> predicted positives: 876 / 2861 (30.62%)


## Predicción final estilo API (limpieza + threshold + salida amigable)

### Qué haremos
Crear una función `predict_api()` que:
1) Recibe texto crudo (lo que mandará backend)
2) Aplica la misma limpieza usada en entrenamiento
3) Calcula probabilidad de positivo
4) Aplica threshold (0.40)
5) Devuelve:
   - prediction: 0/1
   - probability: float
   - label: "positive" / "negative"

### Por qué
Esto deja una única fuente para el comportamiento del modelo,y es lo mismo
que luego se replica en FastAPI.



In [ ]:
CHOSEN_THRESHOLD = 0.40

def predict_with_threshold(model, clean_texts, threshold=CHOSEN_THRESHOLD):
    """
    clean_texts: lista[str] ya LIMPIOS (texto_clean)
    """
    probs = model.predict_proba(clean_texts)[:, 1]
    preds = (probs >= threshold).astype(int)
    return preds, probs

def predict_from_raw(model, raw_texts, threshold=CHOSEN_THRESHOLD):
    """
    raw_texts: lista[str] CRUDOS (texto_de_review o texto recibido por API)
    Internamente los convierte a texto_clean
    """
    clean_texts = [clean_tweet(t) for t in raw_texts]  # usa la función ya existente en tu notebook
    preds, probs = predict_with_threshold(model, clean_texts, threshold)
    return preds, probs, clean_texts


In [ ]:
import joblib, json

MODEL_PATH = "sentiment_pipeline_balanced.joblib"
joblib.dump(pipeline_bal, MODEL_PATH)

with open("threshold.txt", "w") as f:
    f.write(str(CHOSEN_THRESHOLD))

config = {
    "model_path": MODEL_PATH,
    "threshold": CHOSEN_THRESHOLD,
    "labels": {"positive": 1, "neutral_or_negative": 0},
    "pipeline": {
        "tfidf": {"max_features": 5000, "ngram_range": [1, 2], "stop_words": "english"},
        "logreg": {"class_weight": "balanced", "max_iter": 1000}
    }
}

with open("model_config.json", "w") as f:
    json.dump(config, f, indent=2)

print("✅ Exportado:", MODEL_PATH, "threshold.txt", "model_config.json")


✅ Exportado: sentiment_pipeline_balanced.joblib threshold.txt model_config.json


In [ ]:
from google.colab import files

files.download("sentiment_pipeline_balanced.joblib")
files.download("threshold.txt")
files.download("model_config.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>